In [2]:
!pip install -q -U trax
import trax

     |████████████████████████████████| 629 kB 8.4 MB/s 
     |████████████████████████████████| 4.3 MB 40.8 MB/s 
     |████████████████████████████████| 152 kB 60.1 MB/s 
     |████████████████████████████████| 249 kB 71.8 MB/s 
     |████████████████████████████████| 1.2 MB 55.1 MB/s 
     |████████████████████████████████| 2.6 MB 56.9 MB/s 
     |████████████████████████████████| 366 kB 63.8 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 4.0 MB 50.7 MB/s 
     |████████████████████████████████| 895 kB 35.6 MB/s 
     |████████████████████████████████| 636 kB 46.3 MB/s 
     |████████████████████████████████| 3.3 MB 31.8 MB/s 


In [3]:
import os
import trax
import trax.fastmath.numpy as np
import pickle
import numpy
import random as rnd
from trax import fastmath
from trax import layers as tl
import tensorflow as tf

rnd.seed(32)

In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

# change to working tensorflow directory on the drive
#%cd '/content/gdrive/My Drive/models/'

Mounted at /content/gdrive


In [32]:
%cd '/content/gdrive/My Drive/Hafez'

/content/gdrive/My Drive/Hafez


In [50]:

Vocab={'':0,'__EOS__':1,'__UNK__':2}

lines=[]

with open('./all_ghazals.txt',encoding="utf8") as f:

    for ln in f:
        #print(ln)

        if ln.strip():
            #print(ln)
            if '\u200c' in ln:
                continue
            ln = ln.strip('\n')
            if len(ln.replace(" ",""))<10:
                #print('not word')
                #print(ln)
                continue

            else:
                lines.append(ln)
                for c in ln:

                    if c not in Vocab:

                        Vocab[c]=len(Vocab)


print(Vocab)
print(len(Vocab))
print(lines[:10])
print(len(lines))


{'': 0, '__EOS__': 1, '__UNK__': 2, 'ا': 3, 'ل': 4, ' ': 5, 'ي': 6, 'ه': 7, 'س': 8, 'ق': 9, 'ی': 10, 'د': 11, 'ر': 12, 'ک': 13, 'و': 14, 'ن': 15, 'م': 16, 'ز': 17, 'ج': 18, 'چ': 19, 'ع': 20, 'ش': 21, 'ب': 22, 'گ': 23, 'ت': 24, 'پ': 25, 'غ': 26, 'خ': 27, 'آ': 28, 'ص': 29, 'ح': 30, 'ف': 31, 'ظ': 32, 'ط': 33, 'ث': 34, 'ض': 35, 'ذ': 36, 'ژ': 37, 'Y': 38}
39
['الا يا ايها الساقی ادر کاسا و ناولها', 'مرا در منزل جانان چه امن عيش چون هر دم', 'به می سجاده رنگين کن گرت پير مغان گويد', 'شب تاريک و بيم موج و گردابی چنين هايل', 'همه کارم ز خود کامی به بدنامی کشيد آخر', 'متی ما تلق من تهوی دع الدنيا و اهملها', 'صلاح کار کجا و من خراب کجا', 'ببين تفاوت ره کز کجاست تا به کجا', 'دلم ز صومعه بگرفت و خرقه سالوس', 'کجاست دير مغان و شراب ناب کجا']
7029


In [8]:
eval_lines=lines[-1000:]
lines=lines[:-1000]
print(f"Number of lines for training: {len(lines)}")
print(f"Number of lines for validation: {len(eval_lines)}")

Number of lines for training: 6029
Number of lines for validation: 1000


In [9]:
def line_to_tensor(line,vocab_dict, EOS_int=1):
    """Turns a line of text into a tensor

    Args:
        line (str): A single line of text.
        EOS_int (int, optional): End-of-sentence integer. Defaults to 1.

    Returns:
        list: a list of integers (unicode values) for the characters in the `line`.
    """
    
    # Initialize the tensor as an empty list
    tensor = []
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    # for each character:
    for c in line:

        if c in vocab_dict:

          c_int = vocab_dict[c]

        else:

          c_int = vocab_dict['__UNK__']

        
     
        
        
        # append the unicode integer to the tensor list
        tensor.append(c_int)
    
    # include the end-of-sentence integer
    tensor.append(EOS_int)
    ### END CODE HERE ###

    return tensor

In [10]:
print('the sentence is: \n',lines[0])
print('tensor is :',line_to_tensor(lines[0],Vocab,EOS_int=1))

print('tensor is :',line_to_tensor('lines[0]',Vocab,EOS_int=1))

the sentence is: 
 الا يا ايها الساقی ادر کاسا و ناولها
tensor is : [3, 4, 3, 5, 6, 3, 5, 3, 6, 7, 3, 5, 3, 4, 8, 3, 9, 10, 5, 3, 11, 12, 5, 13, 3, 8, 3, 5, 14, 5, 15, 3, 14, 4, 7, 3, 1]
tensor is : [2, 2, 2, 2, 2, 2, 2, 2, 1]


In [11]:
def data_generator(batch_size, max_length, data_lines, vocab_dict,line_to_tensor=line_to_tensor, shuffle=True):
    """Generator function that yields batches of data

    Args:
        batch_size (int): number of examples (in this case, sentences) per batch.
        max_length (int): maximum length of the output tensor.
        NOTE: max_length includes the end-of-sentence character that will be added
                to the tensor.  
                Keep in mind that the length of the tensor is always 1 + the length
                of the original line of characters.
        data_lines (list): list of the sentences to group into batches.
        line_to_tensor (function, optional): function that converts line to tensor. Defaults to line_to_tensor.
        shuffle (bool, optional): True if the generator should generate random batches of data. Defaults to True.

    Yields:
        tuple: two copies of the batch (jax.interpreters.xla.DeviceArray) and mask (jax.interpreters.xla.DeviceArray).
        NOTE: jax.interpreters.xla.DeviceArray is trax's version of numpy.ndarray
    """
    # initialize the index that points to the current position in the lines index array
    index = 0
    
    # initialize the list that will contain the current batch
    cur_batch = []
    
    # count the number of lines in data_lines
    num_lines = len(data_lines)
    
    # create an array with the indexes of data_lines that can be shuffled
    lines_index = [*range(num_lines)]
    
    # shuffle line indexes if shuffle is set to True
    if shuffle:
        rnd.shuffle(lines_index)
    
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    while True:
        
        # if the index is greater or equal than to the number of lines in data_lines
        if index>=num_lines:
            # then reset the index to 0
            index = 0
            # shuffle line indexes if shuffle is set to True
            if shuffle:
                rnd.shuffle(lines_index)
            
        # get a line at the `lines_index[index]` position in data_lines
        line = data_lines[lines_index[index]]
        
        # if the length of the line is less than max_length
        if len(line)<max_length:
            # append the line to the current batch
            cur_batch.append(line)
            
        # increment the index by one
        index += 1
        
        # if the current batch is now equal to the desired batch size
        if len(cur_batch)==batch_size:
            
            batch = []
            mask = []
            
            # go through each line (li) in cur_batch
            for li in cur_batch:
                # convert the line (li) to a tensor of integers
                tensor = line_to_tensor(li,vocab_dict)
                
                # Create a list of zeros to represent the padding
                # so that the tensor plus padding will have length `max_length`
                pad = [0] * (max_length-len(tensor))
                
                # combine the tensor plus pad
                tensor_pad = tensor+pad
                
                # append the padded tensor to the batch
                batch.append(tensor_pad)

                # A mask for  tensor_pad is 1 wherever tensor_pad is not
                # 0 and 0 wherever tensor_pad is 0, i.e. if tensor_pad is
                # [1, 2, 3, 0, 0, 0] then example_mask should be
                # [1, 1, 1, 0, 0, 0]
                # Hint: Use a list comprehension for this
                example_mask = [1]*len(tensor)+[0]*len(pad)
              
                mask.append(example_mask)
               
            # convert the batch (data type list) to a trax's numpy array
            batch_np_arr = np.array(batch)
            mask_np_arr = np.array(mask)
            
            ### END CODE HERE ##
            
            # Yield two copies of the batch and mask.
            yield batch_np_arr, batch_np_arr, mask_np_arr
            
            # reset the current batch to an empty list
            cur_batch = []

In [12]:
tmp_data_gen = data_generator(batch_size=2, 
                              max_length=64, 
                              data_lines=lines[:10],vocab_dict=Vocab,
                              shuffle=False)

# get one batch
import itertools
infinite_data_generator = itertools.cycle(tmp_data_gen)
tmp_batch = next(tmp_data_gen)

# view the batch
tmp_batch

(DeviceArray([[ 3,  4,  3,  5,  6,  3,  5,  3,  6,  7,  3,  5,  3,  4,  8,
                3,  9, 10,  5,  3, 11, 12,  5, 13,  3,  8,  3,  5, 14,  5,
               15,  3, 14,  4,  7,  3,  1,  0,  0,  0,  0,  0,  0,  0,  0,
                0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
                0,  0,  0,  0],
              [16, 12,  3,  5, 11, 12,  5, 16, 15, 17,  4,  5, 18,  3, 15,
                3, 15,  5, 19,  7,  5,  3, 16, 15,  5, 20,  6, 21,  5, 19,
               14, 15,  5,  7, 12,  5, 11, 16,  1,  0,  0,  0,  0,  0,  0,
                0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
                0,  0,  0,  0]], dtype=int32),
 DeviceArray([[ 3,  4,  3,  5,  6,  3,  5,  3,  6,  7,  3,  5,  3,  4,  8,
                3,  9, 10,  5,  3, 11, 12,  5, 13,  3,  8,  3,  5, 14,  5,
               15,  3, 14,  4,  7,  3,  1,  0,  0,  0,  0,  0,  0,  0,  0,
                0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
                0,  0

In [13]:
def GRULM(vocab_size=256, d_model=512, n_layers=2, mode='train'):
    """Returns a GRU language model.

    Args:
        vocab_size (int, optional): Size of the vocabulary. Defaults to 256.
        d_model (int, optional): Depth of embedding (n_units in the GRU cell). Defaults to 512.
        n_layers (int, optional): Number of GRU layers. Defaults to 2.
        mode (str, optional): 'train', 'eval' or 'predict', predict mode is for fast inference. Defaults to "train".

    Returns:
        trax.layers.combinators.Serial: A GRU language model as a layer that maps from a tensor of tokens to activations over a vocab set.
    """
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    model = tl.Serial(
      tl.ShiftRight(n_positions=1,mode='train'), # Stack the ShiftRight layer
      tl.Embedding(vocab_size,d_model), # Stack the embedding layer
      [tl.GRU(n_units=d_model) for _ in range(n_layers)], # Stack GRU layers of d_model units keeping n_layer parameter in mind (use list comprehension syntax)
      tl.Dense(n_units=d_model), # Dense layer
      tl.LogSoftmax() # Log Softmax
    )
    ### END CODE HERE ###
    return model

In [ ]:
# testing your model
model = GRULM(vocab_size=len(Vocab), d_model=512, n_layers=2, mode='train')
print(model)

Serial[
  Serial[
    ShiftRight(1)
  ]
  Embedding_39_512
  GRU_512
  GRU_512
  Dense_512
  LogSoftmax
]


In [14]:
### Training
batch_size = 32
max_length = 64

In [15]:
def n_used_lines(lines, max_length):
    '''
    Args: 
    lines: all lines of text an array of lines
    max_length - max_length of a line in order to be considered an int
    Return:
    n_lines -number of efective examples
    '''

    n_lines = 0
    for l in lines:
        if len(l) <= max_length:
            n_lines += 1
    return n_lines

num_used_lines = n_used_lines(lines, 64)
print('Number of used lines from the dataset:', num_used_lines)
print('Batch size (a power of 2):', int(batch_size))
steps_per_epoch = int(num_used_lines/batch_size)
print('Number of steps to cover one epoch:', steps_per_epoch)

Number of used lines from the dataset: 6029
Batch size (a power of 2): 32
Number of steps to cover one epoch: 188


In [16]:
from trax.supervised import training

def train_model(model, data_generator, vocab_dict,batch_size=32, max_length=64, lines=lines, eval_lines=eval_lines, n_steps=1, output_dir='model/'): 
    """Function that trains the model

    Args:
        model (trax.layers.combinators.Serial): GRU model.
        data_generator (function): Data generator function.
        batch_size (int, optional): Number of lines per batch. Defaults to 32.
        max_length (int, optional): Maximum length allowed for a line to be processed. Defaults to 64.
        lines (list, optional): List of lines to use for training. Defaults to lines.
        eval_lines (list, optional): List of lines to use for evaluation. Defaults to eval_lines.
        n_steps (int, optional): Number of steps to train. Defaults to 1.
        output_dir (str, optional): Relative path of directory to save model. Defaults to "model/".

    Returns:
        trax.supervised.training.Loop: Training loop for the model.
    """
    
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    bare_train_generator =  data_generator(batch_size=batch_size, max_length=max_length, data_lines=lines,vocab_dict=vocab_dict)
    infinite_train_generator = itertools.cycle(bare_train_generator)
    
    bare_eval_generator = data_generator(batch_size=batch_size, max_length=max_length, data_lines=eval_lines,vocab_dict=vocab_dict)
    infinite_eval_generator = itertools.cycle(bare_eval_generator)
   
    train_task = training.TrainTask(
        labeled_data=infinite_train_generator, # Use infinite train data generator
        loss_layer=tl.CrossEntropyLoss(),   # Don't forget to instantiate this object
        optimizer=trax.optimizers.Adam(learning_rate=0.0005),     # Don't forget to add the learning rate parameter
        n_steps_per_checkpoint=50
    )

    eval_task = training.EvalTask(
        labeled_data=infinite_eval_generator,    # Use infinite eval data generator
        metrics=[tl.CrossEntropyLoss(), tl.Accuracy()], # Don't forget to instantiate these objects
        n_eval_batches=3      # For better evaluation accuracy in reasonable time
    )
    
    training_loop = training.Loop(model,
                                  train_task,
                                  eval_tasks=eval_task,
                                  output_dir=output_dir)

    training_loop.run(n_steps=n_steps)  ##each step equal to a 1 batch
    
    ### END CODE HERE ###
    
    # We return this because it contains a handle to the model, which has the weights etc.
    return training_loop

In [ ]:
#help(trax.supervised.TrainTask)

In [ ]:
#help(trax.supervised.training.Loop)

In [ ]:
epoch=50
training_loop = train_model(GRULM(), data_generator,Vocab,n_steps=steps_per_epoch*epoch)
#training_loop = train_model(GRULM(), data_generator,Vocab,n_steps=steps_per_epoch*epoch)

/usr/local/lib/python3.7/dist-packages/jax/lib/xla_bridge.py:374: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  "jax.host_id has been renamed to jax.process_index. This alias "
/usr/local/lib/python3.7/dist-packages/jax/lib/xla_bridge.py:387: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  "jax.host_count has been renamed to jax.process_count. This alias "



Step   5650: Ran 50 train steps in 101.10 secs
Step   5650: train CrossEntropyLoss |  1.08835423
Step   5650: eval  CrossEntropyLoss |  2.04271452
Step   5650: eval          Accuracy |  0.46328283


KeyboardInterrupt: ignored

In [41]:
def test_model(preds, target):
    """Function to test the model.

    Args:
        preds (jax.interpreters.xla.DeviceArray): Predictions of a list of batches of tensors corresponding to lines of text.
        target (jax.interpreters.xla.DeviceArray): Actual list of batches of tensors corresponding to lines of text.

    Returns:
        float: log_perplexity of the model.
    """
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    total_log_ppx = np.sum(preds *tl.one_hot(target,n_categories=preds.shape[-1]), axis= -1) # HINT: tl.one_hot() should replace one of the Nones

    non_pad = 1.0 - np.equal(target, 0)          # You should check if the target equals 0
    ppx = total_log_ppx * non_pad                             # Get rid of the padding

    log_ppx = np.sum(ppx) / np.sum(non_pad)
    ### END CODE HERE ###
    
    return -log_ppx

In [43]:
model = GRULM()
model.init_from_file('model/model.pkl.gz')
batch = next(data_generator(batch_size, max_length, lines, Vocab,shuffle=False))
print(batch[0].shape)
preds = model(batch[0])
log_ppx = test_model(preds, batch[1])
print('The log perplexity and perplexity of your model are respectively', log_ppx, np.exp(log_ppx))

(32, 64)
The log perplexity and perplexity of your model are respectively 1.0888716 2.9709198


In [44]:
inv_Vocab = {v: k for k, v in Vocab.items()}
inv_Vocab

{0: '',
 1: '__EOS__',
 2: '__UNK__',
 3: 'ا',
 4: 'ل',
 5: ' ',
 6: 'ي',
 7: 'ه',
 8: 'س',
 9: 'ق',
 10: 'ی',
 11: 'د',
 12: 'ر',
 13: 'ک',
 14: 'و',
 15: 'ن',
 16: 'م',
 17: 'ز',
 18: 'ج',
 19: 'چ',
 20: 'ع',
 21: 'ش',
 22: 'ب',
 23: 'گ',
 24: 'ت',
 25: 'پ',
 26: 'غ',
 27: 'خ',
 28: 'آ',
 29: 'ص',
 30: 'ح',
 31: 'ف',
 32: 'ظ',
 33: 'ط',
 34: 'ث',
 35: 'ض',
 36: 'ذ',
 37: 'ژ',
 38: 'Y'}

In [45]:
def gumbel_sample(log_probs, temperature=1.0):
    """Gumbel sampling from a categorical distribution."""
    u = numpy.random.uniform(low=1e-6, high=1.0 - 1e-6, size=log_probs.shape)
    g = -np.log(-np.log(u))
    return np.argmax(log_probs + g * temperature, axis=-1)

def predict(num_chars, prefix):
    inp = [ord(c) for c in prefix]
    result = [c for c in prefix]
    max_len = len(prefix) + num_chars
    for _ in range(num_chars):
        cur_inp = np.array(inp + [0] * (max_len - len(inp)))
        outp = model(cur_inp[None, :])  # Add batch dim.
        next_char = gumbel_sample(outp[0, len(inp)])
        inp += [int(next_char)]
       
        if inp[-1] == 1:
            break  # EOS
        result.append(inv_Vocab[int(next_char)])
    
    return "".join(result)

print(predict(64, ""))

دوش بخت هر کم برنه بدنامی کرده شنيد


In [ ]:
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))

ز گفت بخت جوی کلک عشقياض بلاکر
خم ابروی تو تعويی فرخ پگياريست
من گفتا بگذشتم ای مفلس گفريبد کی شاه
گوش از ما مست تو مهر عنان بپوش
حافظ به رقيب و شربلبی و عطر گذار
هيچ الست شيخ شد و آب رويم
تر مبروی در فضلی هر چه ز گست بنشان هوا
نالحم علل آفريا مده و يار دگر


In [ ]:
#Poet samples:

print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))
print(predict(48, ""))

شب جگر دهد در شمش حافظ ای صبح
زقد عشق تو هيچ رو نه خال تو بست
که درداغ علم و افغان را چه شود جان
در هصطشتان خرقه حافظا نهره بار
از زمانش دهان عمل بفرما که منعمان
شکفته ما گذری هيتی غير منمی
بس شکرتو روی فرخ پی فرخنده پياله بود
هر بخت عدد که در خانگيست در فرزهان
درکش اندر سر فرخ پی خات و هيچ را
بلال خدا که پرتوان شمع دل است
گفت خاتی به راه طريقت گذر نماند
به فقل رای جين کرم بی فری داور کنم
و از می خور و خون دلم راه در مذهب آباد
در کمر خانقاه و حريفان دهان
فراغت آن آن را از بيا و بهر خدا بکن
حافظ مريد فشان خود است حالش
گر خاک بر سر خود خون که تهباری کند
جم خون و تدبير نثارم و به انداز افتض
رضش عشق که گفت بسی تو با خاک بجاست
که طاله قصد جمال داود و نياز
ز رنج عالم تيز عنبرقف اليفان
زبان ز شربلب چه جان طلغمت همه حافظ
واعظ ما چو لبلای از دست ما فرعنگيست
که آنپرسم ژهايت از سر بر نداده يار
چو حال دجر تو را چه گوش خور بهشت
شرم بده عشق کرشمه چين به سر آرد
غرض ابروی دل که در آن گرفت چو مکن
کجا بيمويت عشقبازان را چه سلطل
گريه چه خون دل چهره حورت رخ خوبی
لنگی دريبا به گيسوی حرام افتاد
چو هفتاد باد

In [47]:
## More samples

print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))

علاج بيلوده مژده اما دل درخرفت
کت خود که پرفشان چو چراغ ديده شکايان
هر چند بر صنگ و ز شمشير شد عجب خواند
ترک و حيل دريبا به سر نماز من است
يا رب اين تقايی در آن نرآيد کاين شهاب
گر باد آمده چاک خواب نخواهد لافی
با حافظ و از غم مسغول در خم چوگان
آن گه پيام تا خصم بی چنان حافظ
تا سر گوش به جبرنه از آن که در طلب گير
يا رند نقش عشقی خدای روزن دريغ
چو نگرد علم شادی شود عزيز است
خيز تا از چو نی حور باده به عياری
غلام دهارت کرم بج عالق است مناب
بنجه حافظ از دل ميخانه پای فان
جمال اعتماد و شفا بخوان است ولی
بوی شرف دلا محتسب شهر حور و حرمان است
بی لاله ميروخ تو بدرده بر خضک
بر عزم تو خود دادی که صبا بر باری داند
بی طلبت يار من آرش که چابدان هر دم
سود و دانم که به دست نشنادم
همچو گل گفته و بلکن مرامت رسيد
از بخت من او خندان به خروشد نگارا
آن کسان که من جز فرمانده عشق
غلام دم کز وصل تو تدبير کماند


In [48]:

print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))

فلک خاطر مجموع هوساز و نداد
که من از پرتشمه نفس رفتنج ابرو
آه صافيت شعر دانست پرويی چون ای دل کشيد
منت هر آن يک جهان نقاب و رندی نيست
ما که صوفی ز مدحت انجم يار شما
فراغت مداون که مبادا خلوتی حافظ
پس اگر به صيادت گفتم که سفله گفت
به پياله صيام که کند قامتی دار
چون من سايه آب و رحمت نبود غماز
وگر تربلی گفتنم همناز من و تو باد
حباب و کرم سوی فرخ پی کف ساقی گيرنم
به قدت رکن مانکبش او به جان خواند
از رخ زرق به خون آشفته شود
حقوق صحبت گوهی که از ما بگويمت
ز افتاک هر ديده حافظ خور نماندست
در سينه جام جهان افتاد بلند
جهان نشود روانی به تماشا بچنان افتادست
عهدثه شيرينان و حريف نباشد
در اين خيالت ای ساقی و ابروی ما چند
بت سر عهد ملک دی و سحر گذران
ديوان و نزدين شاه عشقباجان را
بزم در آخر به بام رها ببرد و کنوم
جای ديوانه ياد قدم و من زمانی چو شمع
گفت آن کش نيست زمانی دولت بنده مقصر تو


In [49]:

print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))

print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))
print(predict(64, ""))

گر نرسد ز مصلحت نيخت که ما را خواهد
بر اين پرده نگه دار زدم ديده شاد
لنگر شيرين و بيماری هجر تو زد از نفس
رحم اگر چه و سوخته دل راه و نگار
اين چه سفله که رخصی و روی می خورده باد
صبر خويش بر زلف سوختگان بازآيد
فدای که آرت از اعغر تو همان مشنيست
تا جان کی به باد غم از شما داد و برآر
همچو ار بدمندم صرف ببر کز نماند
به خنده گفتم بی چو طلعت خويشم
ماهرزانی همنشين دارم ز من باشد
که باد آن کس که هزارش مرحال افتاد
سمن سبک و دختر دار کمن شويم
هر آن کس که می خور شدی به من بزد ولی
بشنو که خواب خوش ديدار شمابخش
فغان که روز که باده پرستان همه فال
خواهی از آشنا منور از سر نمازی
حافظ ببر و وفا نيست جای حافظ
برکليد لعت تو تنگ در چين زمانی
مژده در غزل خرابات است مجلق کرد
عشقبازان را خلافيت شاهانه زد و سول
که داخت برخيز کرد و تمکان بازرسان
که در پای خم ابروا ياد شاه
که آخر به يک ديده حافظ عزيز امل
هر دست به دستم نديدم رسيد امان
گر طريق عشف فتضه از فلاصل دوران غير
خوشدلی ای نصيب تو در هزاران رواست
با يار مفلس شراب آلوده
وين بشنويد رندان پاکداريم
جلوه و خرد وصل کنم به باده دهد
بحر ميان خسروان ما را بخورد ن